In [12]:
# 1. Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [13]:
# 2. Đọc file csv và gắng index với giá Price
df = pd.read_csv('BNB.csv',sep=";")
df1=df.reset_index()['Price']

In [14]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [15]:
# 4. Chia train test với tỉ kệ 5:3:2
train_size = int(0.5 * len(df1))
test_size = int(0.3 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

In [16]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [17]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [18]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [19]:
# 8. Define LSTM Model
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [20]:
# 9. Fit mô hình với dữ liệu train
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
13/13 [==============================] - 12s 425ms/step - loss: 1.5294e-04 - val_loss: 0.2030
Epoch 2/100
13/13 [==============================] - 3s 231ms/step - loss: 6.0799e-05 - val_loss: 0.0463
Epoch 3/100
13/13 [==============================] - 3s 234ms/step - loss: 2.4887e-05 - val_loss: 0.0325
Epoch 4/100
13/13 [==============================] - 3s 229ms/step - loss: 1.5668e-05 - val_loss: 0.0253
Epoch 5/100
13/13 [==============================] - 4s 353ms/step - loss: 1.3644e-05 - val_loss: 0.0221
Epoch 6/100
13/13 [==============================] - 3s 230ms/step - loss: 1.3438e-05 - val_loss: 0.0212
Epoch 7/100
13/13 [==============================] - 3s 231ms/step - loss: 1.2513e-05 - val_loss: 0.0216
Epoch 8/100
13/13 [==============================] - 3s 225ms/step - loss: 1.3677e-05 - val_loss: 0.0186
Epoch 9/100
13/13 [==============================] - 4s 334ms/step - loss: 1.1062e-05 - val_loss: 0.0173
Epoch 10/100
13/13 [==============================] - 

In [21]:
# 10. Dự báo dữ liệu test, val
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)
y_pred_val=model.predict(X_val)


9/9 [==============================] - 1s 62ms/step


In [22]:
# 11. Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [23]:
# 13. Dự báo 30 ngày tiếp theo
x_input=val_data[60:].reshape(1,-1)
x_input.shape

temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=305
i=0
while(i<30):

    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1

0 day input [0.38700077 0.3929528  0.37762633 0.38833998 0.38344444 0.37896554
 0.38238795 0.35738944 0.36423427 0.39741682 0.40574966 0.43016785
 0.42137373 0.4151241  0.41510922 0.4155705  0.43595619 0.45678829
 0.46333552 0.46201119 0.47360276 0.47687638 0.47598357 0.4825308
 0.47479317 0.4813404  0.47553717 0.46573121 0.46869234 0.463648
 0.44994346 0.43671508 0.40860663 0.4155705  0.4422207  0.43997381
 0.43835188 0.43431939 0.44161062 0.40875543 0.40664246 0.40469317
 0.41869532 0.41214809 0.40859175 0.40740135 0.40619606 0.40664246
 0.40829415 0.40502053 0.38357836 0.40814535 0.41066008 0.42985537
 0.43463187 0.43283138 0.43046545 0.40604726 0.40902327 0.39652402
 0.40232724 0.40856199 0.38939646 0.39801202 0.38939646 0.38683709
 0.40292245 0.40515446 0.40188084 0.40115172 0.40412773 0.39860723
 0.41229689 0.4158681  0.41719243 0.41424618 0.41647819 0.42033212
 0.43417059 0.43089697 0.42033212 0.41350217 0.40441045 0.40710374
 0.3972829  0.3969853  0.3962413  0.39756562 0.394292

In [24]:
# Respliting train test
train_size1 = int(0.5 * len(df))
test_size1 = int(0.3 * len(df))
train_data1 = df[:train_size1]
test_data1 = df[train_size1:train_size1+test_size1]
val_data1 = df[train_size1+test_size1:]
# Repredictions on the testing set
x_test1 = np.array(test_data1.drop('Price', axis=1))  # Use the features (excluding the target variable)
y_test1 = np.array(test_data1['Price'])

# Repredictions on the validation set
x_val1 = np.array(val_data1.drop('Price', axis=1))  # Use the features (excluding the target variable)
y_val1 = np.array(val_data1['Price'])

In [25]:
#Calculate evalution
#RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - y_val1)**2))
test_rmse = np.sqrt(np.mean((y_pred - y_test1)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

#MAPE
valid_mape = np.mean(np.abs((y_val1 - y_pred_val) / y_val1)) * 100
test_mape = np.mean(np.abs((y_test1 - y_pred) / y_test1)) * 100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

#MAE
valid_mae = np.mean(np.abs(y_pred_val - y_val1))
test_mae = np.mean(np.abs(y_pred - y_test1))
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

Validation RMSE: 49.24297154371739
Testing RMSE: 186.3427490515217
Validation MAPE: 13.93154662906872
Testing MAPE: 160.02849955935946
Validation MAE: 41.43802233529281
Testing MAE: 149.2229217075413
